<a href="https://colab.research.google.com/github/Rompil/Advanced-CV/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! gdown --id 1VSIMAR3-2fXTEy-QdY2d0M_-aC1aXfWp

In [ ]:
!unzip -q Classification_data.zip
!ls

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
image_size = (150, 150)
batch_size = 32

In [ ]:


train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "Classification_data/train",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "Classification_data/test",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

In [ ]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

In [ ]:
def make_simple_model(input_shape, num_classes):
    inputs=keras.Input(shape=input_shape)
    # Image augmentation block
    x = data_augmentation(inputs)
    x = layers.experimental.preprocessing.Rescaling(1.0/255)(x)
    x = layers.Conv2D(32, 3,  padding="same")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Activation("relu")(x)
    x = layers.Flatten()(x)
    x = layers.Dense(100, activation="relu")(x)
    x = layers.Dropout(0.2)(x)

    activation = "softmax"
    units = num_classes
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)
model = make_simple_model(input_shape=image_size + (3,), num_classes=6)
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
epochs = 23

callbacks = [
    keras.callbacks.ModelCheckpoint("/content/drive/MyDrive/Colab Notebooks/models/simple_save_at_{epoch}.h5"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss= tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

In [ ]:
def make_advanced_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = data_augmentation(inputs)

    # Entry block
    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)

    activation = "softmax"
    units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model = make_advanced_model(input_shape=image_size + (3,), num_classes=6)
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
epochs = 50

callbacks = [
    keras.callbacks.ModelCheckpoint("/content/drive/MyDrive/Colab Notebooks/models/advanced_save_at_{epoch}.h5"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss= tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model.save("/content/drive/MyDrive/Colab Notebooks/models/final_model")

In [ ]:
img = keras.preprocessing.image.load_img(
    "/content/Classification_data/test/sea/20072.jpg", target_size=image_size
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

model.load()
predictions = model.predict(img_array)
score = predictions[0]
print(
    predictions
)


VGG16 as a base architecture

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
vgg = VGG16(include_top=False, weights='imagenet', input_shape=list(image_size) + [3])

In [ ]:
# freeze some layers
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
# new layers
x = layers.Flatten()(vgg.output)
x = layers.Dense(1000, activation='relu')(x)
prediction = layers.Dense(6, activation='softmax')(x)

In [ ]:
# create a model object
model = keras.models.Model(inputs=vgg.input, outputs=prediction)

In [ ]:
x = np.concatenate([x for x, y in val_ds], axis=0)
y = np.concatenate([y for x, y in val_ds], axis=0)

In [ ]:
 def is_correct_prediction(prediction, true_value):
     print(np.argmax(prediction).shape)
     print(true_value.shape)
     return true_value == np.argmax(prediction)



prediction = model.predict(x)
arg_max = np.argmax(prediction, axis=1)
(arg_max == y)
# print(prediction.shape)
# is_correct_prediction(model.predict(x), y)

In [ ]:
np.sum(arg_max == y)

In [ ]:
def top5_best(x, y, model):
    prediction = model.predict(x)
    match = []
    mismatch = []
    print(is_correct_prediction(prediction, y))

x = np.concatenate([x for x, y in val_ds], axis=0)
y = np.concatenate([y for x, y in val_ds], axis=0)
top5_best(x, y, model)

In [ ]:
import numpy as np

In [ ]:
x = np.concatenate([x for x, y in val_ds], axis=0)
y = np.concatenate([y for x, y in val_ds], axis=0)
y.shape

In [ ]:

x = [(model.predict(x[0]), y) for x, y in train_ds]

In [ ]:
x = [x for x, _ in val_ds]
model.predict(x)